## Working with bits



* To utilize certain features of microcontrollers, often times we need to turn on or off certain bits of certain memory location.



* The pointer of C/C++ is a suitable tool for this type of task.



* However, the smallest unit of memory that we can change is a byte; hence we may need to first read the current status of the particular byte, change that one bit, and then write the *bit pattern* to that byte.



## 16 bit color example



* IBM's [XGA](https://en.wikipedia.org/wiki/Graphics_display_resolution) graphic card standard in 1990s was the first with 1024 &times; 768 resolution. Its 640 &times; 480 mode could use 16 bit [*high color*](https://en.wikipedia.org/wiki/High_color).



* To represent red, green, and blue colors with 15 bits, 5 bits per each would be the natural choice.



* With 16 bits, 1 more bit for green; hence red 5, green 6, and blue 5 bits.



In [ ]:
# https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook
# http://nbviewer.jupyter.org/github/ipython/ipython/blob/4.0.x/examples/IPython%20Kernel/Rich%20Output.ipynb

import IPython.display as disp

# number of bits
n = 16
nr = 5
ng = 6
nb = 5

disp.display(
    disp.Markdown(
        '\n'.join(
            [
                ' | '.join(str(k) for k in range(n-1, 0-1, -1)),
                '|'.join(':---:' for k in range(1, n+1)),
                ' | '.join(['`b`']*nb + ['`g`']*ng + ['`r`']*nr),
            ], # Prepare a list of table rows
        ) # Join all rows with new line characters in between
    ) # Create a markdown object with the string for table
) # Present the markdown table



* But why 1 more bit for green?
* Why three color channels from the beginning?



* Anyways, let's see if we can pack three colors in 16 bits.



### Revisiting bitwise operators



* Both C/C++ and python has a number of [bitwise operators](https://en.wikipedia.org/wiki/Bitwise_operation).



| operator | python | C/C++ |
|:-----:|:-----:|:-----:|
| `<<` | `1 << 5` |  `1 << 5` |
| `>>` | `int('10000', base=2) & >> 4` |  `0x10 >> 4` |
| `&` | `int('11101011', base=2) & int('11111', base=2)` |  `235 << 31` |
| <code>&vert;</code> | 0 <code>&vert; (1 << 7)</code> |  `0 << 128` |
| `&=` | `a = 10; a &= int('0111', base=2)` |  `char a = 10; a &= 0b0111;` |



#### 16bit color using bitwise operator



* Let's try to implement 16bit color using the bitwise operators.



In [ ]:
%%writefile bitwise_16bit_color.cpp
#include <bitset>
#include <cstdint>
#include <cstdlib>
#include <iomanip>
#include <iostream>


int32_t main(const int32_t argn, const char * argv[]){

    // number of examples
    const int32_t n = 10;

    // table header
    std::cout << "| `r` | `g` | `b` | `hex` |" << '\n';
    std::cout << "|:---:|:---:|:---:|:-----:|" << '\n';

    uint32_t i = 0;
    // example loop
    for(i = 0; n > i; ++i){
        // variables for r g b colors and index
        uint32_t r = rand() % (1 << 5);
        uint32_t g = rand() % (1 << 6);
        uint32_t b = rand() % (1 << 5);
        uint32_t hex = 0;

        // set red value from bit 0 ~ 4
        hex |= (r & 0x1F);
        
        // set green value from bit 5 ~ 10
        hex |= ((g & 0x3F) << 5);
        
        // set blue value from bit 11 ~ 15
        hex |= ((b & 0x1F) << 11);
        
        // How hex is supposed to have 16bit color value

        // bit patterns of color values
        // https://stackoverflow.com/questions/7349689
        std::bitset<5> b_r(r);
        std::bitset<6> b_g(g);
        std::bitset<6> b_b(b);

        // bit pattern of the 16bit integer
        std::bitset<16> b_hex(hex);

        // print this example as a row
        std::cout << "| " << std::hex << std::setw(6) << b_r
                    << " | " << std::setw(6) << b_g
                    << " | " << std::setw(6) << b_b
                    << " | " << std::setw(16) << b_hex
                    << " |\n";
    }

}



In [ ]:
%%bash
# The directive above would run following as bash commands

# Detect OS type because OSX may need different options
# https://stackoverflow.com/questions/3466166/how-to-check-if-running-in-cygwin-mac-or-linux/18790824

# obtain system information
unameOut="$(uname -s)"

# detect system type
case "${unameOut}" in
    Linux*)     machine=Linux;;
    Darwin*)    machine=Mac;;
    CYGWIN*)    machine=Cygwin;;
    MINGW*)     machine=MinGw;;
    *)          machine="UNKNOWN:${unameOut}"
esac

if [ $machine == "Linux" ]; then
    # build command for Linux
    g++ -Wall -g bitwise_16bit_color.cpp -o ./bitwise_16bit_color -Wa,-adhln=bitwise_16bit_color.s
elif [ "Mac" == $machine ]; then
    # build command for OSX
    # https://stackoverflow.com/questions/10990018/
    clang++ -S -mllvm --x86-asm-syntax=intel bitwise_16bit_color.cpp
    clang++ -Wall -g bitwise_16bit_color.cpp -o bitwise_16bit_color
else
    # Otherwise
    g++ -Wall -g bitwise_16bit_color.cpp -o ./bitwise_16bit_color.s -S
    g++ -Wall -g bitwise_16bit_color.cpp -o ./bitwise_16bit_color
fi



In [ ]:
# https://stackoverflow.com/questions/4760215/running-shell-command-from-python-and-capturing-the-output
# https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook
import subprocess
import IPython.display as disp

# Run executable while capturing output
result = subprocess.run(['./bitwise_16bit_color'], stdout=subprocess.PIPE)

# present output as a markdown table
disp.display(disp.Markdown(result.stdout.decode()))



In [ ]:
%%bash
# Run bash command to delete .cpp and executable files
rm bitwise_16bit_color.cpp bitwise_16bit_color



* Let's take a look at the assembly instructions.



``` 
  24:bitwise_16bit_color.cpp ****         uint32_t hex = 0;
 565                            .loc 6 24 0
 566 00cf C745C400              movl    $0, -60(%rbp)
 566      000000
  25:bitwise_16bit_color.cpp ****
  26:bitwise_16bit_color.cpp **** // set red value from bit 0 ~ 4
  27:bitwise_16bit_color.cpp ****         hex |= (r & 0x1F);
 567                            .loc 6 27 0
 568 00d6 8B45B8                movl    -72(%rbp), %eax
 569 00d9 83E01F                andl    $31, %eax
 570 00dc 0945C4                orl     %eax, -60(%rbp)
  28:bitwise_16bit_color.cpp ****
  29:bitwise_16bit_color.cpp **** // set green value from bit 5 ~ 10
  30:bitwise_16bit_color.cpp ****         hex |= ((g & 0x3F) << 5);
 571                            .loc 6 30 0
 572 00df 8B45BC                movl    -68(%rbp), %eax
 573 00e2 C1E005                sall    $5, %eax
 574 00e5 25E00700              andl    $2016, %eax
 574      00
 575 00ea 0945C4                orl     %eax, -60(%rbp)
  31:bitwise_16bit_color.cpp ****
  32:bitwise_16bit_color.cpp **** // set blue value from bit 11 ~ 15
  33:bitwise_16bit_color.cpp ****         hex |= ((b & 0x1F) << 11);
 576                            .loc 6 33 0
 577 00ed 8B45C0                movl    -64(%rbp), %eax
 578 00f0 C1E00B                sall    $11, %eax
 579 00f3 0FB7C0                movzwl  %ax, %eax
 580 00f6 0945C4                orl     %eax, -60(%rbp)

```



* It seems that the compiler used following [instructions](https://en.wikipedia.org/wiki/X86_instruction_listings) to implement.



| instruction | opcode | expected behavior | example |
|:-----:|:-----:|:-----:|:-----:|
| `movl` | C7 | set a value to a memory location (32bit) | `movl $0, -60(%rbp)` |
| `movl` | 8B | copy a value at a memory location to another (32bit) | `movl -72(%rbp), %eax` |
| `andl` | 83 | bitwise and operation to a memory location (32bit) | `andl $31, %eax` |
| `orl` | 09 | bitwise or operation to a memory location (32bit) | `orl %eax, -60(%rbp)` |
| `sall` | C1 | bitwise left shift to a memory location (32bit) | `sall $5, %eax` |
| `movzwl` | 0F | expand 16 bit value to longer unsigned integer | `movzwl  %ax, %eax` |



* It also seems that the result of the operation remains at the latter of the two operands.



* Let's take a look at how to designate operands: values or memory locations.



| operand | designation | example |
|:-----:|:-----:|:-----:|
| direct value | decimal number | `$0`, `$31` |
| eax register | general purpose register `eax` ([32bit](https://en.wikipedia.org/wiki/X86#32-bit)) | `%eax` |
| ax register | general purpose register `ax` ([16bit](https://en.wikipedia.org/wiki/X86#16-bit)) | `%ax` |
| pointer | memory at address | `-60(%rbp)`, `-72(%rbp)` |
| rbp register | [base pointer](https://en.wikipedia.org/wiki/X86#Purpose) register (64bit) | `%rbp` |



* Now let's look at the first a few lines of the assembly code above.



```
  24:bitwise_16bit_color.cpp ****         uint32_t hex = 0;
 565                            .loc 6 24 0
 566 00cf C745C400000000        movl    $0, -60(%rbp)
```



* The last line is the machine code / assembly translation from the C/C++ code of the first line.



| machine language code | meaning |
|:-----:|:-----:|
| `00cf` | the start location of this machine code |
| `C7` | `movl` immediate value |
| `45` | `(%rbp)` addressing using base pointer |
| `C4` | offset from %rbp<br> $$ 60_{10}=00111100_2 \\ 11000100_2 = C4_{16} $$ |
| `0000 0000` | immediate value to write to `-60(%rbp)` location |



* This line would initialize `uint_32t` variable `hex` with zero.



* Now let's take a look at the next a few lines



```
  25:bitwise_16bit_color.cpp ****
  26:bitwise_16bit_color.cpp **** // set red value from bit 0 ~ 4
  27:bitwise_16bit_color.cpp ****         hex |= (r & 0x1F);
 567                            .loc 6 27 0
 568 00d6 8B45B8                movl    -72(%rbp), %eax
 569 00d9 83E01F                andl    $31, %eax
 570 00dc 0945C4                orl     %eax, -60(%rbp)
```



* The C/C++ line here tries to copy the lower 5 bits of variable `r` to `hex`.



* The first assembly instruction copies a 32 bit value to `eax` register.



* The second instruction applies 32bit bitwise *and* operation to `eax` register.  As $31_{10} = 11111_2$, all bits between 31 ~ 5 would be 0.



* The third instruction carrys out bitwise *or* operation to `uint32_t` variable `hex` with the value of `eax` register.



* THe following lines are similar, too.



```
  28:bitwise_16bit_color.cpp ****
  29:bitwise_16bit_color.cpp **** // set green value from bit 5 ~ 10
  30:bitwise_16bit_color.cpp ****         hex |= ((g & 0x3F) << 5);
 571                            .loc 6 30 0
 572 00df 8B45BC                movl    -68(%rbp), %eax
 573 00e2 C1E005                sall    $5, %eax
 574 00e5 25E0070000            andl    $2016, %eax
 575 00ea 0945C4                orl     %eax, -60(%rbp)
```



* Let's focus on the following two lines.



```
 573 00e2 C1E005                sall    $5, %eax
 574 00e5 25E0070000            andl    $2016, %eax
```



* Here, <br>`sall    $5, %eax`<br>intend to shift the 32bit content of `eax` to the left by five binary digits.



* In the third line is as follows.<br>`andl    $2016, %eax`
* Because <br>$2016_{10} = 07e0_{16}= 0000 0111 1110 0000_2$<br>this would turn off bits of `eax` except 6 bits associated with `g`.
* Compare with C/C++ line.  Is the order same? Why or why not?

